In [1]:
import pandas as pd
import numpy as np

scale = 0.5

# Domain
x = 150*scale
z = 74*scale
y_air = 5*scale
y_moon = 50*scale
y = y_moon + y_air

y_cave = 10
x_co = 0.25
x_step = 0.1
y_gpr = 0.1

# Reglith Layer
y_regolith = 4.5

# Crust Layer
y_crust = y_moon - y_regolith

# Frequency and Wavelength
f_max = 2*100e6 # Maximum Relevant Frequency for the Ricker Wavelet
c = 3e8

# Permittivity and Density
density = 2 # g/cm^3
eps_bg = 1.919**density # Background Permittivity for Moon
vel_bg = c / eps_bg**0.5

lambda_0 = c / (f_max * eps_bg**0.5)

# # Fresnel Zone - Radius
# R = 0.5 * (lambda_0 * d)**0.5
# y = 4 * R

# Grid Descretization
dx = np.round(lambda_0 / 17.3, 3) * 2
dy = np.round(lambda_0 / 17.3, 3) * 2
dz = np.round(lambda_0 / 17.3, 3) * 2

nx = int(x / dx)
ny = int(y / dy)
ny_m = int(y_moon / dy)
nz = int(z / dz)
print(f"nx: {nx}, ny: {ny}, nz: {nz}")

# x_grid, y_grid, z_grid = np.meshgrid(x_vals, y_vals, z_vals, indexing='xy')

# Time Window
# time_window = 2 * y_moon / vel_bg
# time_window = 1e-6 # For Highland Basalt
time_window = 6e-7 # For Mare Basalt

# Print
print(f"Domain: {x} x {y} x {z}")
print(f"Base Permittivity of Moon: {eps_bg}")
print(f"Frequency: {f_max/2}")
print(f"Wavelength: {lambda_0}")
# print(f"Fresnel Zone Radius: {R}")
print(f"Grid: {dx} x {dy} x {dz}")
print(f"Time Window: {time_window}")

nx: 833, ny: 305, nz: 411
Domain: 75.0 x 27.5 x 37.0
Base Permittivity of Moon: 3.682561
Frequency: 100000000.0
Wavelength: 0.7816571130797291
Grid: 0.09 x 0.09 x 0.09
Time Window: 6e-07


In [2]:
print(f"y_regolith: {y_regolith}")
print(f"y_crust: {y_crust}")
print(f"y_air: {y_air}")
print(f"y_moon: {y_moon}")
print(f"y_cave: {y_cave}")
print(f"x_co: {x_co}")
print(f"x_step: {x_step}")
print(f"y_gpr: {y_gpr}")

y_regolith: 4.5
y_crust: 20.5
y_air: 2.5
y_moon: 25.0
y_cave: 10
x_co: 0.25
x_step: 0.1
y_gpr: 0.1


In [3]:
#read boxes2.csv
df = pd.read_csv(f'boxes_{int(1/scale)}_delaunay.csv')

# Convert the DataFrame to a list of tuples
cave_boxes = [
    ((row['Start X'], row['Start Y'], row['Start Z']),
     (row['End X'], row['End Y'], row['End Z']))
    for index, row in df.iterrows()
]
num_boxes = len(cave_boxes)

In [4]:
print(f"Number of Boxes: {num_boxes}")

Number of Boxes: 91496


In [5]:
with open(f'mare_tube_{int(1/scale)}_B_05_01_delaunay.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {6e-7}\n")

    # Define the GPR source
    f.write(f"#waveform: ricker 1 {f_max/2} ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/10} {y_moon + y_gpr} {z/2} ricker_wavelet\n")
    
    f.write(f"#soil_gen_bruggeman_moon: 13 17.8 2.2 3.2 13.5 15.5 7.5 12.5 40.5 47.5 8.5 12.5 {f_max/2} soil_reg\n")
    f.write(f"#fractal_box: 0 {y_crust} 0 {x} {y_moon} {z} 1.5 1 1 1 25 soil_reg moon_regolith\n")

    f.write(f"#soil_gen_bruggeman_moon: 19 22 1 6 6 10.5 6 11.5 42 47 9.5 10.5 {f_max/2} rocks_crust\n")
    f.write(f"#fractal_box: 0 0 0 {x} {y_crust} {z} 1.5 1 1 1 25 rocks_crust moon_crust\n")

    # Loop over each cell to define the materials
    for i in range(num_boxes):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {cave_boxes[i][0][0]} {cave_boxes[i][0][1]} {cave_boxes[i][0][2]} {cave_boxes[i][1][0]} {cave_boxes[i][1][1]} {cave_boxes[i][1][2]} free_space\n")

    f.write(f"#rx: {x/10 + 2*x_co} {y_moon} {z/2}\n")
    f.write(f"#src_steps: {x_step} 0 0\n#rx_steps: {x_step} 0 0\n")
    f.write(f"geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} mare_tube_{int(1/scale)}_B_05_01 n \n")

In [6]:
with open(f'mare_tube_{int(1/scale)}_B_05_0_delaunay.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {6e-7}\n")

    # Define the GPR source
    f.write(f"#waveform: ricker 1 {f_max/2} ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/10} {y_moon} {z/2} ricker_wavelet\n")
    
    f.write(f"#soil_gen_bruggeman_moon: 13 17.8 2.2 3.2 13.5 15.5 7.5 12.5 40.5 47.5 8.5 12.5 {f_max/2} soil_reg\n")
    f.write(f"#fractal_box: 0 {y_crust} 0 {x} {y_moon} {z} 1.5 1 1 1 25 soil_reg moon_regolith\n")

    f.write(f"#soil_gen_bruggeman_moon: 19 22 1 6 6 10.5 6 11.5 42 47 9.5 10.5 {f_max/2} rocks_crust\n")
    f.write(f"#fractal_box: 0 0 0 {x} {y_crust} {z} 1.5 1 1 1 25 rocks_crust moon_crust\n")

    # Loop over each cell to define the materials
    for i in range(num_boxes):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {cave_boxes[i][0][0]} {cave_boxes[i][0][1]} {cave_boxes[i][0][2]} {cave_boxes[i][1][0]} {cave_boxes[i][1][1]} {cave_boxes[i][1][2]} free_space\n")

    f.write(f"#rx: {x/10 + 2*x_co} {y_moon} {z/2}\n")
    f.write(f"#src_steps: {x_step} 0 0\n#rx_steps: {x_step} 0 0\n")
    f.write(f"geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} mare_tube_{int(1/scale)}_B_05_0 n \n")

In [7]:
with open(f'hland_tube_{int(1/scale)}_B_05_01_delaunay.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write(f"#waveform: ricker 1 {f_max/2} ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/10} {y_moon + y_gpr} {z/2} ricker_wavelet\n")
    
    f.write(f"#soil_gen_bruggeman_moon: 1 10 0.1 3 24 34 1 9.5 41.5 48.5 11 21.5 {f_max/2} soil_reg\n")
    f.write(f"#fractal_box: 0 {y_crust} 0 {x} {y_moon} {z} 1.5 1 1 1 25 soil_reg moon_regolith\n")

    f.write(f"#soil_gen_bruggeman_moon: 5 10.01 0.4 1.59 17.9 28 6.9 12.5 44.4 48.4 11.1 15 {f_max/2} rocks_crust\n")
    f.write(f"#fractal_box: 0 0 0 {x} {y_crust} {z} 1.5 1 1 1 25 rocks_crust moon_crust\n")

    # Loop over each cell to define the materials
    for i in range(num_boxes):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {cave_boxes[i][0][0]} {cave_boxes[i][0][1]} {cave_boxes[i][0][2]} {cave_boxes[i][1][0]} {cave_boxes[i][1][1]} {cave_boxes[i][1][2]} free_space\n")

    f.write(f"#rx: {x/10 + 2*x_co} {y_moon} {z/2}\n")
    f.write(f"#src_steps: {x_step} 0 0\n#rx_steps: {x_step} 0 0\n")
    f.write(f"geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} hland_tube_{int(1/scale)}_B_05_01 n \n")

In [8]:
with open(f'hland_tube_{int(1/scale)}_B_05_0_delaunay.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write(f"#waveform: ricker 1 {f_max/2} ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/10} {y_moon} {z/2} ricker_wavelet\n")
    
    f.write(f"#soil_gen_bruggeman_moon: 1 10 0.1 3 24 34 1 9.5 41.5 48.5 11 21.5 {f_max/2} soil_reg\n")
    f.write(f"#fractal_box: 0 {y_crust} 0 {x} {y_moon} {z} 1.5 1 1 1 25 soil_reg moon_regolith\n")

    f.write(f"#soil_gen_bruggeman_moon: 5 10.01 0.4 1.59 17.9 28 6.9 12.5 44.4 48.4 11.1 15 {f_max/2} rocks_crust\n")
    f.write(f"#fractal_box: 0 0 0 {x} {y_crust} {z} 1.5 1 1 1 25 rocks_crust moon_crust\n")

    # Loop over each cell to define the materials
    for i in range(num_boxes):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {cave_boxes[i][0][0]} {cave_boxes[i][0][1]} {cave_boxes[i][0][2]} {cave_boxes[i][1][0]} {cave_boxes[i][1][1]} {cave_boxes[i][1][2]} free_space\n")

    f.write(f"#rx: {x/10 + 2*x_co} {y_moon} {z/2}\n")
    f.write(f"#src_steps: {x_step} 0 0\n#rx_steps: {x_step} 0 0\n")
    f.write(f"geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} hland_tube_{int(1/scale)}_B_05_0 n \n")